In [13]:
#%%
import openml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

# Get dataset by ID
dataset = openml.datasets.get_dataset(31)
# Convert to pandas dataframe
dataset = pd.DataFrame(dataset.get_data()[0])
dataset_encoded = pd.DataFrame()

In [12]:
dataset['foreign_worker'] = pd.Series(np.where(dataset['foreign_worker'] == 'yes', 1, 0),dataset.index)
dataset['own_telephone'] = pd.Series(np.where(dataset['own_telephone'] == 'yes', 1, 0),dataset.index)
dataset['job_encoded'] = LabelEncoder().fit_transform(dataset['job'])

dataset.head()
# Encode using https://www.kaggle.com/getting-started/27270

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class,job_encoded
0,<0,6,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4,male single,none,...,67,none,own,2,skilled,1,1,1,good,1
1,0<=X<200,48,existing paid,radio/tv,5951.0,<100,1<=X<4,2,female div/dep/mar,none,...,22,none,own,1,skilled,1,0,1,bad,1
2,no checking,12,critical/other existing credit,education,2096.0,<100,4<=X<7,2,male single,none,...,49,none,own,1,unskilled resident,2,0,1,good,3
3,<0,42,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2,male single,guarantor,...,45,none,for free,1,skilled,2,0,1,good,1
4,<0,24,delayed previously,new car,4870.0,<100,1<=X<4,3,male single,none,...,53,none,for free,2,skilled,2,0,1,bad,1


In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(dataset.corr(), annot=True,cmap="YlGnBu")